In [1]:
# In a Jupyter Notebook cell
%run main.py --modelname dab_detr --coco_path ./coco_dataset_test --output_dir ./output_model


c:\Users\Rahman\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Not using distributed mode
[07/27 19:55:45.470]: git:
  sha: N/A, status: clean, branch: N/A

[07/27 19:55:45.470]: Command: main.py --modelname dab_detr --coco_path ./coco_dataset_test --output_dir ./output_model
[07/27 19:55:45.471]: Full config saved to ./output_model\config.json
[07/27 19:55:45.472]: world size: 1
[07/27 19:55:45.472]: rank: 0
[07/27 19:55:45.473]: local_rank: 0
[07/27 19:55:45.474]: args: Namespace(lr=0.0001, lr_backbone=1e-05, batch_size=4, weight_decay=0.0001, epochs=100, lr_drop=200, save_checkpoint_interval=100, clip_max_norm=0.1, modelname='dab_detr', frozen_weights=None, backbone='resnet50', dilation=False, position_embedding='sine', pe_temperatureH=20, pe_temperatureW=20, batch_norm_type='FrozenBatchNorm2d', return_interm_layers=False, backbone_freeze_keywords=None, enc_layers=6, dec_layers=6, dim_feedforward=2048, hidden_dim=256, dropout=0.3, nheads=8, num_queries=300, pre_norm=False, num_select=300, transformer_activation='prelu', num_patterns=0, random_r

In [ ]:
import os, sys
import torch
import numpy as np

from models import build_DABDETR
from util.slconfig import SLConfig
from datasets import build_dataset
from util.visualizer import COCOVisualizer
from util import box_ops

We provide this notebook for inference and visualizations. 

You can either load images from a dataloader(see Sec. 1) or load an image from a local path(see Sec. 2).

# 0. Init and Load Pre-trained Models

In [ ]:
model_config_path = "./pre_train_model/config.json" # change the path of the model config file
model_checkpoint_path = "./output_model/checkpoint.pth" # change the path of the model checkpoint
# See our Model Zoo section in README.md for more details about our pretrained models.


In [ ]:
args = SLConfig.fromfile(model_config_path) 
model, criterion, postprocessors = build_DABDETR(args)
checkpoint = torch.load(model_checkpoint_path, map_location='cpu')
model.load_state_dict(checkpoint['model'], strict=False)

# 1. Visualize images from dataloader
## 1.1 Load Datasets

In [ ]:
dataset_train = build_dataset(image_set='train', args=args)
dataset_val = build_dataset(image_set='val', args=args)
cocojs = dataset_val.coco.dataset
id2name = {item['id']: item['name'] for item in cocojs['categories']}

# 1.2 Get an Example and Visualize it

In [ ]:
image, targets = dataset_val[0]

In [ ]:
# build gt_dict for vis
box_label = [id2name[int(item)] for item in targets['labels']]
gt_dict = {
    'boxes': targets['boxes'],
    'image_id': targets['image_id'],
    'size': targets['size'],
    'box_label': box_label,
}
vslzr = COCOVisualizer()
vslzr.visualize(image, gt_dict, savedir=None)

## 1.3 Visualize Model Predictions

In [ ]:
output = model(image[None])
output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]))[0]

In [ ]:
thershold = 0.3 # set a thershold

scores = output['scores']
labels = output['labels']
boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
select_mask = scores > thershold

In [ ]:
box_label = [id2name[int(item)] for item in labels[select_mask]]
pred_dict = {
    'boxes': boxes[select_mask],
    'size': targets['size'],
    'box_label': box_label
}
vslzr.visualize(image, pred_dict, savedir=None)

# 2. Visualize Custom Images

In [ ]:
from PIL import Image
import datasets.transforms as T

In [ ]:
image = Image.open("./figure/idea.jpg").convert("RGB")
# image

In [ ]:
# transform images
transform = T.Compose([
    T.RandomResize([800], max_size=1333),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
image, _ = transform(image, None)

In [ ]:
# predict images
output = model(image[None])
output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]))[0]

In [ ]:
# visualize outputs
thershold = 0.3 # set a thershold

scores = output['scores']
labels = output['labels']
boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
select_mask = scores > thershold

box_label = [id2name[int(item)] for item in labels[select_mask]]
pred_dict = {
    'boxes': boxes[select_mask],
    'size': torch.Tensor([image.shape[1], image.shape[2]]),
    'box_label': box_label
}
vslzr.visualize(image, pred_dict, savedir=None)